In [6]:
import numpy as np
import numpy.random as random
import pandas as pd
import scipy as sp
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

%precision 3


'%.3f'

In [7]:
import sklearn

In [9]:
import requests, zipfile
import io

url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

auto.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
                'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
                'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
                'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [10]:
print('자동차 데이터 형태:{}'.format(auto.shape))

자동차 데이터 형태:(205, 26)


In [12]:
auto.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [13]:
#물음표 개수 확인

auto = auto[['price', 'horsepower','width','height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [14]:
#NaN으로 변환 후 제거

auto = auto.replace('?', np.nan).dropna()
print('자동차 데이터 형태:{}'.format(auto.shape))

자동차 데이터 형태:(199, 4)


In [16]:
print('데이터 형 확인(형 변환 전)\n{}\n'.format(auto.dtypes))

데이터 형 확인(형 변환 전)
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [18]:
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('데이터 형 확인(형 변환 후)\n{}'.format(auto.dtypes))

데이터 형 확인(형 변환 후)
price           int64
horsepower      int64
width         float64
height        float64
dtype: object


In [19]:
auto.corr()

#상관관계가 높은 변수들을 다중회귀의 모델에 함께 설명변수로 투입하면 다중공선성(multicollinearity)이 발생할 가능성이 있음
#변수 간의 상관관계가 높아 회귀계수의 분산이 커지고, 결과적으로 회귀계수의 통계적 유의성이 없어짐
#이러한 이유로 상관관계가 높은 변수 그룹을 대표하는 변수만을 다중회귀모델에 투입함

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


In [22]:
#데이터 분할(훈련 데이터와 테스트 데이터)을 위한 임포트
from sklearn.model_selection import train_test_split

#다중회귀 모델 구축을 위한 임포트
from sklearn.linear_model import LinearRegression

#목표변수로 price를 지정, 설명변수로는 price 외의 다른 변수를 지정
X = auto.drop('price', axis=1)
y = auto['price']

#훈련 데이터와 테스트 테이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

#다중회귀 클래스 초기화와 학습
model = LinearRegression()
model.fit(X_train, y_train)

#결정계수 출력
print('결정계수(train):{:.3f}'.format(model.score(X_train, y_train)))
print('결정계수(test):{:.3f}'.format(model.score(X_test, y_test)))

print('\n회귀계수\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('절편: {:.3f}'.format(model.intercept_))

결정계수(train):0.733
결정계수(test):0.737

회귀계수
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
절편: -128409.046
